<a href="https://colab.research.google.com/github/Andrii-Radyhin/Andrii-Radyhin/blob/main/Workplace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Dataset -> https://www.kaggle.com/datasets/rajkumarl/- people-clothing-segmentation
- Architecture -> EfficientnetB3 (320,300)
- loss -> BCE
- optimizer -> Adam
- Required OpenCV catalyst Pytorch numpy matplotlib pandas

#Config

In [ ]:
IMG_SIZE = (320, 300) 
crop={'x_min': 115,
      'y_min': 210,
      'x_max': 350,
      'y_max':445}

In [1]:
import cv2
import torch
import numpy as np
import os

In [2]:
# Opens the inbuilt camera of laptop to capture video.
directory='/content/framess2'
path='/content/drive/MyDrive/Camera 3_20220526_003249.mp4'

cap = cv2.VideoCapture(path)
i = 1

os.chdir(directory)

j=0
while(cap.isOpened()):
    ret, frame = cap.read()
     
    # This condition prevents from infinite looping
    # incase video ends.
    if ret == False:
        break
     
    # Save Frame by Frame into disk using imwrite method
    if (i % 1000) == 0:
      cv2.imwrite(str(j)+'.jpg', frame)
      j+=1
    i += 1
 
cap.release()
cv2.destroyAllWindows()

Opencv+numpy 

In [ ]:
import os
from os import listdir
from torchvision.transforms import ToTensor
from torchvision import transforms

images=np.array()

folder_dir = "/kaggle/png_images"

for img in os.listdir(folder_dir):
  img = cv2.imread(img)

  crop_img = img[crop['y_min'] : crop['y_max'],
                 crop['x_min'] : crop['x_max']]

  #normalize in range[0,1]
  crop_img = (crop_img-min(crop_img))/(max(crop_img)-min(crop_img)) 
  images.append(crop_img)

images = ToTensor(images)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
images = normalize(images)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, target, test_size=0.33, random_state=42, shuffle=True)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
dataset_train = TensorDataset(X_train, y_train)
dataset_test = TensorDataset(X_test, y_test)
loaders = {
    "train": DataLoader(dataset_train, batch_size=32, num_workers=1),
    "valid": DataLoader(dataset_test, batch_size=32, num_workers=1)
    }

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import torchvision.models as models
import torch

model = models.efficientnet_b3(pretrained=True)
model.train()
model.classifier[1]=torch.nn.Linear(in_features=1536, out_features=1, bias=True)

model=model.to(device)

In [ ]:
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

In [ ]:
batch_size = 10

for epoch in range(5000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = model(x_batch)['out']
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = model(X_test)['out']
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

In [6]:
15/5

3.0